# Glassdoor analysis

In [36]:
# import requests 
# r = requests.get( 
# 'http://api.glassdoor.com/api/api.htmv=1&format=json&t.p=120&t.k=fz6JLNDfgVs&action=employers&q=pharmaceuticals&userip=192.168.43.42&useragent=Mozilla/%2F4.0' 
# ) 
# response_JSON = r.json() 

In [37]:
# !pip install selenium

In [50]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By

In [51]:
'''Gathers pharmaceutical companies as a dataframe, scraped from Glassdoor'''    
#Initializing the webdriver
options = webdriver.ChromeOptions()

#Uncomment the line below if you'd like to scrape without a new Chrome window every time.
#options.add_argument('headless')

#Change the path to where chromedriver is in your home folder.
driver = webdriver.Chrome(executable_path="https://github.com/arnaud-dg/Glassdoor_Analysis/blob/main/chromedriver.exe", options=options)
driver.set_window_size(1120, 1000)

url = 'https://www.glassdoor.fr/Avis/index.htm?overall_rating_low=0&page=1&sector=10005&filterType=RATING_OVERALL'
driver.get(url)
companies = []

C:\Users\pharm\AppData\Local\Temp\ipykernel_33104\4071902041.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="https://github.com/arnaud-dg/Glassdoor_Analysis/blob/main/chromedriver.exe", options=options)


In [52]:
# Recherche du nombre d'entreprises référencées
# time.sleep(4)
# nombre_entreprises = driver.find_element(By.XPATH, '//*[@class="resultCount css-56kyx5"]')

In [53]:
companies = []

# Boucle de récupération des noms d'entreprises
while len(companies) < 50:
    time.sleep(4)
    
    entreprises = driver.find_element(By.TAG_NAME, 'h2')
    liste_entreprise = entreprises.find_elements(By.XPATH, '//*[@data-test="employer-short-name"]')
    for e in liste_entreprise:
        companies.append(e.text)
        print(e.text)
    
    driver.find_element(By.XPATH, '//*[@class="SVGInline-svg navIcon-svg css-g9i05x-svg e13qs2073-svg"]').click()

Sanofi
Johnson & Johnson
IQVIA
GSK
Novartis
Roche
Eurofins Scientific
Bayer
Thermo Fisher Scientific
Pfizer


ElementClickInterceptedException: Message: element click intercepted: Element <svg class="SVGInline-svg navIcon-svg css-g9i05x-svg e13qs2073-svg" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24">...</svg> is not clickable at point (722, 738). Other element would receive the click: <button id="onetrust-accept-btn-handler">...</button>
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F98893+48451]
	(No symbol) [0x00F2B8A1]
	(No symbol) [0x00E35058]
	(No symbol) [0x00E64BA4]
	(No symbol) [0x00E636E8]
	(No symbol) [0x00E61EEB]
	(No symbol) [0x00E612FE]
	(No symbol) [0x00E59AAC]
	(No symbol) [0x00E7A2BC]
	(No symbol) [0x00E59586]
	(No symbol) [0x00E7A614]
	(No symbol) [0x00E8C482]
	(No symbol) [0x00E7A0B6]
	(No symbol) [0x00E57E08]
	(No symbol) [0x00E58F2D]
	GetHandleVerifier [0x011F8E3A+2540266]
	GetHandleVerifier [0x01238959+2801161]
	GetHandleVerifier [0x0123295C+2776588]
	GetHandleVerifier [0x01022280+612144]
	(No symbol)

In [35]:
companies

['Sanofi',
 'Johnson & Johnson',
 'IQVIA',
 'GSK',
 'Novartis',
 'Roche',
 'Eurofins Scientific',
 'Bayer',
 'Thermo Fisher Scientific',
 'Pfizer',
 'Sanofi',
 'Johnson & Johnson',
 'IQVIA',
 'GSK',
 'Novartis',
 'Roche',
 'Eurofins Scientific',
 'Bayer',
 'Thermo Fisher Scientific',
 'Pfizer',
 'Sanofi',
 'Johnson & Johnson',
 'IQVIA',
 'GSK',
 'Novartis',
 'Roche',
 'Eurofins Scientific',
 'Bayer',
 'Thermo Fisher Scientific',
 'Pfizer',
 'Sanofi',
 'Johnson & Johnson',
 'IQVIA',
 'GSK',
 'Novartis',
 'Roche',
 'Eurofins Scientific',
 'Bayer',
 'Thermo Fisher Scientific',
 'Pfizer',
 'Sanofi',
 'Johnson & Johnson',
 'IQVIA',
 'GSK',
 'Novartis',
 'Roche',
 'Eurofins Scientific',
 'Bayer',
 'Thermo Fisher Scientific',
 'Pfizer',
 'Sanofi',
 'Johnson & Johnson',
 'IQVIA',
 'GSK',
 'Novartis',
 'Roche',
 'Eurofins Scientific',
 'Bayer',
 'Thermo Fisher Scientific',
 'Pfizer',
 'Sanofi',
 'Johnson & Johnson',
 'IQVIA',
 'GSK',
 'Novartis',
 'Roche',
 'Eurofins Scientific',
 'Bayer',
 'Th

In [ ]:
# jobs.append({"Job Title" : job_title,
#             "Salary Estimate" : salary_estimate,
#             "Job Description" : job_description,
#             "Rating" : rating,
#             "Company Name" : company_name,
#             "Location" : location,
#             "Headquarters" : headquarters,
#             "Size" : size,
#             "Founded" : founded,
#             "Type of ownership" : type_of_ownership,
#             "Industry" : industry,
#             "Sector" : sector,
#             "Revenue" : revenue,
#             "Competitors" : competitors})
#             #add job to jobs

#         #Clicking on the "next page" button
#         try:
#             driver.find_element_by_xpath('.//li[@class="next"]//a').click()
#         except NoSuchElementException:
#             print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
#             break

#     return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [ ]:
def get_companies(keyword, companies, verbose):
    
    '''Gathers pharmaceutical companies as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="https://github.com/arnaud-dg/Glassdoor_Analysis/blob/main/chromedriver.exe", options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.fr/Avis/index.htm?overall_rating_low=0&page=1&sector=10005&filterType=RATING_OVERALL'
    driver.get(url)
    companies = []

    while len(companies) < companies:

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.ID, 'id-search-field')
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [ ]:
df = get_jobs("data scientist", 5, False)
df